## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/phonocardiogram/data/PV/present"

In [4]:
waves = np.load(path + "/present_PV_waves_10sec.npy", allow_pickle= True)


In [5]:
print(waves.shape)


(1198, 40000)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [8]:
lbs.feature.zero_crossing_rate(waves[7],frame_length= 8000, hop_length = 12000)

array([[0.01375 , 0.032625, 0.032   , 0.033125]])

In [9]:
lbs.feature.zero_crossing_rate(waves[7], frame_length= 8000, hop_length = 12000).shape

(1, 4)

In [10]:
def zero_crossing(array, frame_length= 2048, hl = 512):
    zero_crossing_rate = np.ones(shape = (array.shape[0], 5)) #buradaki ikinci sayı hop lenght e göre değişmeli 

    for i in range(array.shape[0]):
        zero_crossing_rate[i][0:4] = lbs.feature.zero_crossing_rate(array[i],  frame_length = frame_length,
                                                                    hop_length = hl)
        zero_crossing_rate[i][4] = sum(lbs.zero_crossings(array[i]))
    return zero_crossing_rate

        

In [11]:
zcrs = zero_crossing(waves, 8000, hl = 12000) #çok uzun sürüyor, ya datayı kısaltmalıyız ya da hop lenght artmalı

In [12]:
type(zcrs[0])

numpy.ndarray

In [13]:
np.save(path + "/present_PV_zcrs_array", zcrs, allow_pickle= True)

In [14]:
columns_zcr = []
for i in range(zcrs.shape[1]-1):
    columns_zcr.append("zcrs{}".format(i))

    
columns_zcr.append("#ofcrossing")

print(columns_zcr)

['zcrs0', 'zcrs1', 'zcrs2', 'zcrs3', '#ofcrossing']


In [15]:
zcrs[7]

array([1.3750e-02, 3.2625e-02, 3.2000e-02, 3.3125e-02, 1.2220e+03])

In [23]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[0]):
        llist = list(array[i][0:l-1].tolist())
        llist.append(array[i][l-1])
        
        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [17]:
df = dataframe_creator(zcrs, l =5,  columns = columns_zcr)

df.reset_index(drop = True, inplace = True)

In [18]:
df.head()

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing
0,0.016000,0.028875,0.029625,0.046125,1387.0
1,0.024750,0.034750,0.033375,0.029875,1411.0
2,0.014625,0.032750,0.025875,0.032125,1189.0
3,0.011875,0.033000,0.022000,0.043750,1266.0
4,0.014250,0.046625,0.047625,0.047125,1685.0


In [19]:
df.to_csv(path + '/present_PV_zcrs.csv')

## with Özgün Ozan Nacitarhan code

In [20]:
def zcr(array):
    zcr = np.ones(shape = (array.shape[0],1))
    for i in range(array.shape[0]):
        data_zeros = np.zeros(len(array[i]))
        data_zeros[1:] = array[i][:len(array[i]) - 1]
        zcr[i][0] = (1 / (2 * len(array[i]))) * np.sum(np.abs(np.sign(array[i]) - np.sign(data_zeros)))
        
    return zcr


In [21]:
zcrs_özgün = zcr(waves)

In [22]:
zcrs_özgün.shape

(1198, 1)

In [23]:
zcrs_özgün[0]

array([0.0346625])

In [24]:
np.save(path + "/present_PV_zcrs_özgün", zcrs_özgün, allow_pickle= True)

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import librosa.display

In [27]:
lbs.feature.chroma_stft(waves[0], sr = 4000, hop_length = 12000).shape

(12, 4)

In [28]:
# we are taking time average of each pitch values.
def chromagram_average(array, sampling_rate = 256):
    pitch = np.ones(shape = (array.shape[0], 12)) 
    
    for i in range(array.shape[0]):
        for j in range(12):
            pitch[i][j] = lbs.feature.chroma_stft(np.array(array[i]), sr= sampling_rate, hop_length = 12000)[j].mean()
    return pitch

In [29]:
chro = chromagram_average(waves, sampling_rate = 4000) # bu da çok uzun sürüyor.

In [30]:
type(chro[0])

numpy.ndarray

In [31]:
chro.shape

(1198, 12)

In [32]:
np.save(path + "/present_PV_chromogram_array", chro, allow_pickle= True)

In [33]:
print(chro.shape)

(1198, 12)


In [34]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [35]:
col_chro = []
for i in range(chro.shape[1]):
    col_chro.append(lst[i])
    
print(col_chro)
print(len(col_chro))

['C', 'C♯', 'D', 'D♯', 'E', 'F', 'F♯', 'G', 'G♯', 'A', 'A♯', 'B']
12


In [36]:
ch_df = dataframe_creator(chro,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.361276,0.18809,0.0868,0.138867,0.267305,0.458633,0.546011,0.807571,0.627838,0.361924,0.176646,0.248592
1,0.4076,0.494801,0.558321,0.350664,0.238178,0.273537,0.692079,0.89467,0.451084,0.275629,0.336385,0.347217
2,0.482642,0.461671,0.450648,0.566977,0.637708,0.698479,0.719953,0.67386,0.743078,0.621953,0.476776,0.430923
3,0.238385,0.248205,0.273198,0.329084,0.295685,0.43718,0.703044,0.51067,0.549033,0.684301,0.52347,0.334992
4,0.240413,0.247287,0.432816,0.897314,0.509179,0.293332,0.375806,0.405794,0.579791,0.493779,0.397369,0.260196
...,...,...,...,...,...,...,...,...,...,...,...,...
1193,0.787261,0.866237,0.854227,0.531692,0.457695,0.458081,0.51741,0.51492,0.61011,0.812995,0.834386,0.831439
1194,0.612268,0.83509,0.799304,0.670258,0.696083,0.754348,0.624674,0.548216,0.465375,0.394852,0.404237,0.416524
1195,0.696603,0.744654,0.728119,0.633169,0.689976,0.532535,0.607987,0.608323,0.699683,0.856535,0.882801,0.754319
1196,0.655882,0.696401,0.602679,0.480421,0.448376,0.374977,0.509118,0.61837,0.620731,0.572516,0.750002,0.637642


In [37]:
ch_df.to_csv(path + "/present_PV_chromogram.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [9]:
def tempo_average(array): # 192,79
    means= []
    for i in range(array.shape[0]): #192
        means.append(array[i].mean())
    return np.array(means) # 192
        

In [10]:
def tempogram(array = 0, hop_lenght = 12000, sampling_rate = 256):
    
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sampling_rate, 
                                        lag =3)

    times = librosa.times_like(oenv, sr=sampling_rate, hop_length=12000)

    tempogram = librosa.feature.tempogram(onset_envelope = oenv, sr = sampling_rate,
                                              hop_length = hop_lenght, win_length = 192 )
    # Estimate the global tempo for display purposes
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sampling_rate,
                                   #hop_length=hop_length)[0]
    return tempogram

In [11]:
tempogram(waves[0]).shape

(192, 79)

In [12]:
tempogram(waves[0])[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [13]:
tempogram(waves[0])[1]

array([0.80517355, 0.80649683, 0.80783818, 0.80919789, 0.81057621,
       0.81197334, 0.81338942, 0.81482458, 0.81627887, 0.81775233,
       0.81924491, 0.82075656, 0.82228715, 0.82383652, 0.82540447,
       0.82699073, 0.82859502, 0.830217  , 0.83185629, 0.83351247,
       0.83518509, 0.83687367, 0.83857766, 0.84029653, 0.84202967,
       0.84377649, 0.84553632, 0.84730852, 0.84909239, 0.85088722,
       0.85269229, 0.85450687, 0.85633019, 0.8581615 , 0.86000002,
       0.86184497, 0.86369557, 0.86555103, 0.86741057, 0.86927339,
       0.8711387 , 0.87300574, 0.87487373, 0.8767419 , 0.87860948,
       0.88047575, 0.88233995, 0.88420137, 0.8860593 , 0.88791305,
       0.88976194, 0.89160531, 0.89344251, 0.89527293, 0.89709596,
       0.89891101, 0.90071752, 0.90251494, 0.90430276, 0.90608047,
       0.90784758, 0.90960364, 0.91134822, 0.91308089, 0.91480126,
       0.91650896, 0.91820363, 0.91988495, 0.9215526 , 0.9232063 ,
       0.92484578, 0.92647078, 0.92808108, 0.92967646, 0.93125

In [14]:
tempo_average(tempogram(waves[0]))

array([ 1.00000000e+00,  8.69964397e-01,  7.07931138e-01,  6.51307510e-01,
        6.85073283e-01,  7.56091577e-01,  7.65802804e-01,  7.22457855e-01,
        6.85163267e-01,  6.80484031e-01,  6.92943259e-01,  7.00482673e-01,
        7.00398193e-01,  6.82602871e-01,  6.91599859e-01,  6.91068737e-01,
        6.49323845e-01,  6.11934219e-01,  5.96462921e-01,  6.22443332e-01,
        6.61547149e-01,  6.76250195e-01,  6.46579177e-01,  5.82218488e-01,
        5.62221260e-01,  5.91417058e-01,  6.20715692e-01,  6.26922672e-01,
        5.78383947e-01,  5.22462703e-01,  5.11537060e-01,  5.37342368e-01,
        5.62984159e-01,  5.57118906e-01,  5.17435229e-01,  4.72387509e-01,
        4.60505190e-01,  4.54274434e-01,  4.44609217e-01,  4.43192183e-01,
        4.25061212e-01,  4.18050464e-01,  4.42380547e-01,  4.47954494e-01,
        4.14319487e-01,  3.76328125e-01,  3.70682701e-01,  3.96358982e-01,
        4.04503582e-01,  3.83776030e-01,  3.42623031e-01,  3.23601145e-01,
        3.27847320e-01,  

# tempogram(waves[0])[0]

In [15]:
def temp_values(array):
    final = np.ones(shape = (array.shape[0], 192))
    for i in range(array.shape[0]): #8bin civarı
        
        _x = tempogram(array[i]) #1,4000   #192,79
        #print(_x.shape)
        final[i] = tempo_average(_x) #192
        
        
    return final

In [16]:
temp = temp_values(waves) #çok uzun sürüyor 


In [17]:
type(temp)

numpy.ndarray

In [18]:
np.save(path + "/present_PV_tempogram_array", temp, allow_pickle= True)

In [19]:
temp.shape

(1198, 192)

In [20]:
columns_t = []
for i in range(temp.shape[1]):
    columns_t.append("tempogram_{}".format(i))
print(columns_t)

['tempogram_0', 'tempogram_1', 'tempogram_2', 'tempogram_3', 'tempogram_4', 'tempogram_5', 'tempogram_6', 'tempogram_7', 'tempogram_8', 'tempogram_9', 'tempogram_10', 'tempogram_11', 'tempogram_12', 'tempogram_13', 'tempogram_14', 'tempogram_15', 'tempogram_16', 'tempogram_17', 'tempogram_18', 'tempogram_19', 'tempogram_20', 'tempogram_21', 'tempogram_22', 'tempogram_23', 'tempogram_24', 'tempogram_25', 'tempogram_26', 'tempogram_27', 'tempogram_28', 'tempogram_29', 'tempogram_30', 'tempogram_31', 'tempogram_32', 'tempogram_33', 'tempogram_34', 'tempogram_35', 'tempogram_36', 'tempogram_37', 'tempogram_38', 'tempogram_39', 'tempogram_40', 'tempogram_41', 'tempogram_42', 'tempogram_43', 'tempogram_44', 'tempogram_45', 'tempogram_46', 'tempogram_47', 'tempogram_48', 'tempogram_49', 'tempogram_50', 'tempogram_51', 'tempogram_52', 'tempogram_53', 'tempogram_54', 'tempogram_55', 'tempogram_56', 'tempogram_57', 'tempogram_58', 'tempogram_59', 'tempogram_60', 'tempogram_61', 'tempogram_62', '

In [21]:
len(columns_t)

192

In [24]:
df_t = dataframe_creator(temp,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 192)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_0,tempogram_1,tempogram_2,tempogram_3,tempogram_4,tempogram_5,tempogram_6,tempogram_7,tempogram_8,tempogram_9,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,1.0,0.869964,0.707931,0.651308,0.685073,0.756092,0.765803,0.722458,0.685163,0.680484,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,1.0,0.846305,0.627546,0.532354,0.548772,0.560142,0.535368,0.509864,0.537951,0.625426,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,1.0,0.780674,0.489872,0.397215,0.479685,0.597127,0.594512,0.505375,0.424454,0.456429,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,1.0,0.778969,0.484204,0.360636,0.416983,0.516052,0.522251,0.458615,0.390761,0.412904,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,1.0,0.87089,0.663834,0.548104,0.515456,0.52115,0.521213,0.509559,0.52525,0.562378,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,1.0,0.697405,0.321711,0.291114,0.393916,0.407819,0.369321,0.366829,0.359247,0.346879,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1194,1.0,0.844596,0.605722,0.452775,0.414372,0.44441,0.493617,0.500581,0.4651,0.463384,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1195,1.0,0.821914,0.601569,0.448109,0.381916,0.29665,0.226488,0.227451,0.243146,0.263015,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0
1196,1.0,0.789112,0.487438,0.341567,0.339528,0.368023,0.355097,0.338034,0.332308,0.351293,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [25]:
df_t['tempogram_188'][10]

1.6202653091365476e-17

In [26]:
df_t.to_csv(path + '/present_PV_tempogram.csv')

In [27]:
df_t.isin([0]).sum(axis=0)

tempogram_0      0
tempogram_1      0
tempogram_2      0
tempogram_3      0
tempogram_4      0
                ..
tempogram_187    0
tempogram_188    0
tempogram_189    0
tempogram_190    0
tempogram_191    0
Length: 192, dtype: int64

In [28]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

NameError: name 'df' is not defined

In [56]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,0.016000,0.028875,0.029625,0.046125,1387.0,0.361276,0.18809,0.0868,0.138867,0.267305,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,0.024750,0.034750,0.033375,0.029875,1411.0,0.4076,0.494801,0.558321,0.350664,0.238178,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.014625,0.032750,0.025875,0.032125,1189.0,0.482642,0.461671,0.450648,0.566977,0.637708,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.011875,0.033000,0.022000,0.043750,1266.0,0.238385,0.248205,0.273198,0.329084,0.295685,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.014250,0.046625,0.047625,0.047125,1685.0,0.240413,0.247287,0.432816,0.897314,0.509179,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,0.024375,0.037250,0.028125,0.021750,1237.0,0.787261,0.866237,0.854227,0.531692,0.457695,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1194,0.022875,0.026125,0.027250,0.015125,1170.0,0.612268,0.83509,0.799304,0.670258,0.696083,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1195,0.023250,0.038750,0.025000,0.038625,1696.0,0.696603,0.744654,0.728119,0.633169,0.689976,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0
1196,0.023125,0.024125,0.029875,0.063250,1815.0,0.655882,0.696401,0.602679,0.480421,0.448376,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [57]:
final_df.to_csv(path + '/present_PV_final_sp.csv')